# Homework 2: Association Rule Mining

- **100 points**
- **Due Monday, Nov 10 by 11:59pm**

- *Goals of this homework:* implement the association rule mining process with the Apriori algorithm.

- *Submission instructions:* for this homework, you only need to submit to Blackboard. Please name your submission **FirstName_Lastname_hw2.ipynb**, so for example, my submission would be something like **Md_Mottalib_hw2.ipynb**. Your notebook should be fully executed so that we can see all outputs. 

In this assignment, you are going to examine movies using our understanding of association rules. For this part, you need to implement the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [16]:
import numpy as np
from itertools import combinations
from tqdm.notebook import tqdm
import pandas as pd

user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id watched the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been watched by a single user as a transaction. 

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user movie-watching behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach. 

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will explain the process of the method.**

**Hint: If you implement your algorithm correctly, you should be able to see intermediate result as:**
- Candidates of length 1 count: 408
- After Pruning count: 21
- Candidates of length 2 count: 210
- After Pruning 2 count: 36
- Candidates of length 3 count: 55
- After Pruning 3 count: 12
- Candidates of length 4 count: 1
- After Pruning 4 count: 0

- __load_data__

This function will read a data file, which contains user-movie ratings, and transform it into a "transaction" database. Each transaction will be a list of movies (items) that a single user has rated.

Write the body of the _load_data_ function. It must read the file at file_path and group all movie IDs by their user ID. The final output should be a list where each element is a list of movies rated by a single user.

__Parameters__

file_path (str): The path to the input data file (e.g., "movie_rated.txt").

__Return Value__

list: A list of lists. Each inner list represents a "transaction" (all movies rated by one user).

> ##### Example
>
> If the `movie_rated.txt` file contains:
>
> ```
> 1,101
> 1,102
> 2,101
> 1,103
> 2,104
> ```
>
> The function should return: `[[101, 102, 103], [101, 104]]`
> (Note: The order of users/transactions in the final list does not matter).

In [17]:
import numpy as np
from itertools import combinations

def load_data(file_path):
    """
    Loads data from a specified file path. The file is expected
    to be a .txt file with comma-separated values (user_id, movie_id).
    
    This function should:
    1. Read the file using np.loadtxt.
    2. Group all movies by user ID into a dictionary.
    3. Return a list of the dictionary's values (the movie lists).
    """
    # Read the data file with user_id and movie_id columns
    data = np.loadtxt(file_path, delimiter=',')
    
    # Create a dictionary to group movies by user
    # Key: user_id, Value: list of movie_ids
    user_movies = {}
    
    # Iterate through each row in the data
    for row in data:
        user_id = int(row[0])  # First column is user ID
        movie_id = int(row[1])  # Second column is movie ID
        
        # If user_id not in dictionary, create a new empty list for this user
        if user_id not in user_movies:
            user_movies[user_id] = []
        
        # Append the movie_id to this user's list of movies
        user_movies[user_id].append(movie_id)
    
    # Return list of movie lists (each list is a transaction)
    return list(user_movies.values())

- __generate_candidate__

This function generates a list of candidate itemsets of a specific size _i_. This is a "naive" generator: it finds all unique items across all transactions and then creates all possible combinations of that size.

Write the body of the _generate_candidate_ function. It must find every unique item present in the data and then return all unique combinations of those items of length _i_.

__Parameters__

data (list): A list of transactions (e.g., [[1, 2], [2, 3]]).
i (int): The desired size for the candidate itemsets (e.g., 2).

__Return Value__

list: A list of sets. Each set is a candidate itemset of size i.

> ##### Example
>
> `data = [[1, 2, 5], [2, 3]]`
> 
> `i = 2`
>
> 1.  Unique items are: `{1, 2, 3, 5}`
> 2.  Combinations of size 2 are: `(1, 2), (1, 3), (1, 5), (2, 3), (2, 5), (3, 5)`
> 3.  The function should return: `[{1, 2}, {1, 3}, {1, 5}, {2, 3}, {2, 5}, {3, 5}]`

In [18]:
def generate_candidate(data, i):
    """
    Generates candidate itemsets of length 'i' from the input data.
    
    This function should:
    1. Flatten the input 'data' (list of lists) to find all unique items.
    2. Generate all unique combinations of these items with length 'i'.
    3. Return a list of these combinations, with each combination
       converted to a 'set'.
    """
    # Initialize set to store all unique items
    unique_items = set()
    
    # Check if data contains sets (from previous iterations) or lists (initial data)
    if data and isinstance(list(data)[0], set):
        # Data contains sets from previous iteration
        # Flatten by updating unique_items with each itemset
        for itemset in data:
            unique_items.update(itemset)
    else:
        # Data contains lists (transactions)
        # Flatten by updating unique_items with each transaction
        for transaction in data:
            unique_items.update(transaction)
    
    # Generate all combinations of unique items with length i
    # Sort items for consistent ordering
    candidates = list(combinations(sorted(unique_items), i))
    
    # Convert each tuple to a set and return as list
    return [set(candidate) for candidate in candidates]

- __calculate_support__

This function calculates the support for a single candidate itemset over the entire transaction database. Support is the fraction of transactions that contain the candidate itemset.

Write the body of the _calculate_support_ function. It must count how many transactions in the transactions list contain all items in the candidate set and divide that count by the total number of transactions.

__Parameters__

transactions (list): The list of all transactions (list of lists).
candidate (set): The candidate itemset to check (e.g., {1, 2}).

__Return Value__

float: The support value for the candidate, from 0.0 to 1.0.

> ##### Example
>
> `transactions = [[1, 2, 3], [1, 2, 4], [2, 3, 4], [1, 5]]`
> 
> `candidate = {1, 2}`
>
> 1.  Transaction `[1, 2, 3]` contains `{1, 2}`. (Count = 1)
> 2.  Transaction `[1, 2, 4]` contains `{1, 2}`. (Count = 2)
> 3.  Transaction `[2, 3, 4]` does not contain `{1, 2}`.
> 4.  Transaction `[1, 5]` does not contain `{1, 2}`.
> 5.  Total transactions = 4.
>
> The function should return: `2 / 4 = 0.5`

In [19]:
def calculate_support(transactions, candidate):
    """
    Calculates the support for a single candidate itemset.
    Support = (Number of transactions containing the candidate) / (Total number of transactions)
    
    This function should:
    1. Iterate through each transaction in the 'transactions' list.
    2. Check if the 'candidate' set is a subset of the transaction.
    3. Count how many transactions contain the candidate.
    4. Return the support value (count / total transactions).
    """
    # Initialize counter for transactions containing the candidate
    count = 0
    
    # Iterate through each transaction
    for transaction in transactions:
        # Convert transaction to set for efficient subset comparison
        transaction_set = set(transaction)
        
        # Check if candidate is a subset of this transaction
        # (i.e., all items in candidate appear in the transaction)
        if candidate.issubset(transaction_set):
            count += 1
    
    # Calculate support: proportion of transactions containing the candidate
    total_transactions = len(transactions)
    support = count / total_transactions if total_transactions > 0 else 0
    
    return support

- __filter_candidates_by_support__

This function takes a list of candidate itemsets and "prunes" it, keeping only those that meet the min_support threshold.

Write the body of the _filter_candidates_by_support_ function. It should loop through every candidate, calculate its support, and return a new list containing only the "frequent" candidates.

__Parameters__

transactions (list): The list of all transactions.
candidates (list): A list of candidate itemsets (list of sets).
min_support (float): The minimum support threshold.

__Return Value__

list: A list of frequent itemsets (a subset of the candidates list).

> ##### Example
>
> `transactions = [[1, 2], [1, 3], [1, 4], [1, 2, 3]]`
> 
> `candidates = [{1, 2}, {1, 3}, {1, 4}]`
> 
> `min_support = 0.5`
>
> 1.  Support for `{1, 2}` is `2 / 4 = 0.5`. (Keep)
> 2.  Support for `{1, 3}` is `2 / 4 = 0.5`. (Keep)
> 3.  Support for `{1, 4}` is `1 / 4 = 0.25`. (Drop)
>
> The function should return: `[{1, 2}, {1, 3}]`

In [20]:
def filter_candidates_by_support(transactions, candidates, min_support):
    """
    Filters a list of candidate itemsets.
    
    This function should:
    1. Iterate through each 'candidate' in the 'candidates' list.
    2. Use your 'calculate_support' function to find its support.
    3. If the support is greater than or equal to 'min_support',
       add the candidate to a new list.
    4. Return the new list of frequent itemsets.
    """
    # Initialize empty list to store frequent itemsets
    frequent_itemsets = []
    
    # Iterate through each candidate itemset
    for candidate in candidates:
        # Calculate support for this candidate using our function
        support = calculate_support(transactions, candidate)
        
        # If support meets or exceeds threshold, keep this itemset
        if support >= min_support:
            frequent_itemsets.append(candidate)
    
    # Return only the frequent itemsets
    return frequent_itemsets

- __has_infrequent_subset__

This function implements the "pruning" step of the Apriori principle. The principle states: "If an itemset is frequent, then all of its subsets must also be frequent." This function checks the inverse: "If a candidate has an infrequent subset, it cannot be frequent."

Write the body of the _has_infrequent_subset_ function. It must generate all proper subsets of the candidate and check if each one is present in the all_frequent_itemsets list.

__Parameters__

candidate (set): A candidate itemset of size $k$ (e.g., {1, 2, 3}).
all_frequent_itemsets (list): A list containing all frequent itemsets found in previous iterations (e.g., [{1}, {2}, {3}, {1, 2}]).

__Return__

Valuebool: True if the candidate has an infrequent subset (a subset not in the list). False if all its subsets are frequent (present in the list).

> ##### Example
>
> `candidate = {1, 2, 3}` (size 3)
> `all_frequent_itemsets = [{1}, {2}, {1, 2}, {1, 3}, {2, 5}]`
>
> 1.  Check subsets of size 1:
>     * `{1}`: Is in the list.
>     * `{2}`: Is in the list.
>     * `{3}`: Is **not** in the list.
>
> The function should immediately return: `True`
>
> ---
>
> `candidate = {1, 2, 3}` (size 3)
> `all_frequent_itemsets = [{1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}]`
>
> 1.  Check subsets of size 1: `{1}`, `{2}`, `{3}` are all in the list.
> 2.  Check subsets of size 2:
>     * `{1, 2}`: Is in the list.
>     * `{1, 3}`: Is in the list.
>     * `{2, 3}`: Is in the list.
>
> The function should return: `False`

In [21]:
def has_infrequent_subset(candidate, all_frequent_itemsets):
    """
    Checks if a candidate has any subsets that are not in the
    list of all previously found frequent itemsets.
    (This is the Apriori pruning step).
    
    This function should:
    1. Generate all proper subsets of the 'candidate' (from size 1
       up to size k-1, where k is the size of the candidate).
    2. For each subset, check if it exists in the
       'all_frequent_itemsets' list.
    3. If *any* subset is *not* found in the list, return True immediately.
    4. If all subsets are found, return False.
    
    Hint: `itertools.combinations` is useful here. Remember that
    `all_frequent_itemsets` is a list of sets, so you must
    convert your generated subsets (which will be tuples) into sets
    for comparison.
    """
    # Get the size of the candidate itemset
    k = len(candidate)
    
    # Base case: if candidate has only 1 item, no proper subsets exist
    if k <= 1:
        return False
    
    # Check all proper subsets (sizes from 1 to k-1)
    for subset_size in range(1, k):
        # Generate all subsets of this size from the candidate
        for subset_tuple in combinations(candidate, subset_size):
            # Convert tuple to set for comparison with frequent itemsets
            subset = set(subset_tuple)
            
            # Check if this subset exists in our list of frequent itemsets
            # If not found, this candidate violates the Apriori principle
            if subset not in all_frequent_itemsets:
                return True  # Found an infrequent subset, prune this candidate
    
    # All subsets are frequent, keep this candidate
    return False

- __apriori_algorithm__
   
This is the main driver function that ties everything together. It repeatedly generates candidates, prunes them, and filters them by support, level by level.

Write the body of the _apriori_algorithm_ function. Use the functions you've already defined to build the iterative logic of the algorithm.

__Parameters__

data (list): The list of all transactions.
x (int): The maximum size of itemsets to find (e.g., 4).
min_support (float): The minimum support threshold.

__Return Value__

list: A list containing all frequent itemsets found, from size 1 up to size x.

In [22]:
def apriori_algorithm(data, x, min_support):
    """
    Runs the full Apriori algorithm to find all frequent
    itemsets up to size 'x'.
    
    This function should:
    1. Initialize an empty list 'all_frequent_itemsets' to store results.
    2. Initialize 'frequent_itemsets' (to hold L_k-1) - start with None or [].
    3. Loop for 'i' from 1 up to 'x':
    4.  Generate candidates C_i:
        * If i == 1, call 'generate_candidate' using the raw 'data'.
        * If i > 1, call 'generate_candidate' using 'frequent_itemsets'
            (which holds L_i-1, the frequent sets from the last loop).
    5.  Prune candidates:
        * Create a new list 'frequent_candidates'.
        * Loop through each 'candidate' in your generated list.
        * Call 'has_infrequent_subset'. If it returns False,
            add the 'candidate' to 'frequent_candidates'.
            (Note: for i=1, has_infrequent_subset will correctly return False).
    6.  Filter candidates by support (L_i):
        * Call 'filter_candidates_by_support' on 'frequent_candidates'
            to get the actual frequent itemsets for level 'i'.
    7.  Check for early exit: If the new list of frequent itemsets is empty,
        break the loop.
    8.  Store results:
        * Add the new frequent itemsets to 'all_frequent_itemsets'.
    9. Return 'all_frequent_itemsets'.
    """
    # Step 1: Initialize list to store all frequent itemsets across all levels
    all_frequent_itemsets = []
    
    # Step 2: Initialize variable to hold L_(k-1) from previous iteration
    frequent_itemsets = []
    
    # Step 3: Main loop - process itemsets of increasing size
    for i in range(1, x + 1):
        print(f"\n{'='*50}")
        print(f"Processing Level {i} (itemsets of size {i})")
        print(f"{'='*50}")
        
        # Step 4: Generate candidate itemsets C_i
        if i == 1:
            # First iteration: generate from raw transaction data
            candidates = generate_candidate(data, i)
        else:
            # Subsequent iterations: generate from L_(i-1)
            candidates = generate_candidate(frequent_itemsets, i)
        
        print(f"Candidates of length {i} count: {len(candidates)}")
        
        # Step 5: Prune candidates using Apriori principle
        # Only keep candidates where ALL subsets are frequent
        pruned_candidates = []
        for candidate in candidates:
            # Check if this candidate has any infrequent subset
            if not has_infrequent_subset(candidate, all_frequent_itemsets):
                pruned_candidates.append(candidate)
        
        print(f"After Pruning {i} count: {len(pruned_candidates)}")
        
        # Step 6: Filter by minimum support to get L_i
        frequent_itemsets = filter_candidates_by_support(data, pruned_candidates, min_support)
        
        print(f"Frequent itemsets of length {i}: {len(frequent_itemsets)}")
        
        # Step 7: Early termination check
        if not frequent_itemsets:
            print(f"\nNo frequent itemsets of size {i} found. Stopping algorithm.")
            break
        
        # Step 8: Add L_i to our complete list of frequent itemsets
        all_frequent_itemsets.extend(frequent_itemsets)
    
    # Step 9: Return all frequent itemsets
    print(f"\n{'='*50}")
    print(f"Algorithm Complete!")
    print(f"Total frequent itemsets found: {len(all_frequent_itemsets)}")
    print(f"{'='*50}\n")
    
    return all_frequent_itemsets

> If we run the following code:
>
> `x = 4`
> 
> `min_support = 0.2`
> 
> `data = load_data("data/movie_rated.txt")`
> 
> `frequent_itemsets = apriori_algorithm(data, x, min_support)`
> 
>
> we should have frequent itemsets:
> 
> `[{260}, {377}, {480}, ... {260, 527}, {260, 2987}, ...]`

- __generate_rules__
  
This function takes the list of frequent itemsets and the full transaction record. It generates all possible association rules from those itemsets and filters them based on a minimum confidence threshold.

Write the body of the _generate_rules_ function. You must iterate through every frequent itemset, generate all non-empty proper subsets as antecedents, and check if the resulting rule's confidence meets the threshold.

__Parameters__

record (list): The list of all transactions (list of lists), e.g., [[1, 2, 3], [1, 2]].
frequent_itemsets (list): The list of all frequent itemsets (list of sets) generated by your Apriori function.
min_confidence (float): The minimum confidence threshold (e.g., 0.5).

__Return Value__

list: A list of tuples. Each tuple represents a strong rule and contains two sets: (antecedent, consequent).

> ##### Example
>
> `record = [[1, 2, 3], [1, 2], [1, 3], [2, 3], [1, 2]]` (5 transactions)
> `frequent_itemsets = [{1, 2, 3}]` (We'll just trace this one itemset for simplicity)
> `min_confidence = 0.5`
>
> 1.  **Trace Itemset `{1, 2, 3}`**:
>     * `support({1, 2, 3})` = 1/5 = 0.2
>     * `support({1, 2})` = 3/5 = 0.6
>     * `support({1, 3})` = 2/5 = 0.4
>     * `support({2, 3})` = 2/5 = 0.4
>
> 2.  **Check Rules (i=2, antecedent size 2)**:
>     * Rule: `{1, 2} -> {3}`
>         * Confidence = `support({1, 2, 3}) / support({1, 2})` = 0.2 / 0.6 = 0.33
>         * (0.33 < 0.5, **Drop**)
>     * Rule: `{1, 3} -> {2}`
>         * Confidence = `support({1, 2, 3}) / support({1, 3})` = 0.2 / 0.4 = 0.5
>         * (0.5 >= 0.5, **Keep**)
>     * Rule: `{2, 3} -> {1}`
>         * Confidence = `support({1, 2, 3}) / support({2, 3})` = 0.2 / 0.4 = 0.5
>         * (0.5 >= 0.5, **Keep**)
>
> 3.  **Check Rules (i=1, antecedent size 1)**:
>     * (Rules like `{1} -> {2, 3}` will have even lower confidence, e.g., 0.2 / support({1}) = 0.2 / 0.6 = 0.33. All are dropped.)
>
> The function should return: `[({1, 3}, {2}), ({2, 3}, {1})]`

In [23]:
def generate_rules(record, frequent_itemsets, min_confidence):
    """
    Generates association rules that meet the min_confidence threshold.
    
    This function should:
    1. Iterate through each 'itemset' in the 'frequent_itemsets' list.
    2. For each itemset, generate all possible "splits" into an
       antecedent and a consequent. (e.g., {1, 2, 3} can be split into
       {1} -> {2, 3}, {2} -> {1, 3}, {3} -> {1, 2}, {1, 2} -> {3}, etc.)
    3. For each split, calculate the confidence:
       conf = support(itemset) / support(antecedent)
       (You will need to use your 'calculate_support' function here).
    4. If the calculated confidence is >= 'min_confidence', store the
       rule as a tuple: (antecedent, consequent).
    5. Return the final list of all strong rules.
    """
    # Initialize list to store association rules that meet confidence threshold
    rules = []
    
    print(f"Generating association rules from {len(frequent_itemsets)} frequent itemsets...")
    
    # Iterate through each frequent itemset
    for itemset in frequent_itemsets:
        # Only consider itemsets with at least 2 items
        # (need at least 2 items to create antecedent -> consequent)
        if len(itemset) < 2:
            continue
        
        # Calculate support of the full itemset once (for efficiency)
        itemset_support = calculate_support(record, itemset)
        
        # Generate all possible antecedents (non-empty proper subsets)
        # Try antecedent sizes from 1 to len(itemset)-1
        for antecedent_size in range(1, len(itemset)):
            # Generate all combinations of this size as potential antecedents
            for antecedent_tuple in combinations(itemset, antecedent_size):
                # Convert tuple to set
                antecedent = set(antecedent_tuple)
                
                # The consequent is the remaining items in the itemset
                consequent = itemset - antecedent
                
                # Calculate support of the antecedent
                antecedent_support = calculate_support(record, antecedent)
                
                # Avoid division by zero
                if antecedent_support == 0:
                    continue
                
                # Calculate confidence of the rule: antecedent -> consequent
                # Confidence = P(consequent | antecedent) = support(antecedent U consequent) / support(antecedent)
                confidence = itemset_support / antecedent_support
                
                # If confidence meets threshold, add this rule
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent))
    
    print(f"Total rules with confidence >= {min_confidence}: {len(rules)}\n")
    
    return rules

Use the following code to generate a movie dictionary that maps movie ID to movie names. Then the following function _rules_to_names_ generates formal rules with the movie names.

In [24]:
movies_df = pd.read_csv('movies.csv')
movie_name = []
movie_id = []
for i in range(len(movies_df)):
    movie_name.append(movies_df.iloc[i,1])
    movie_id.append(movies_df.iloc[i,0])
    
movie_dict = dict(zip(movie_id, movie_name))

In [25]:
def rules_to_names(rules, movie_dict):
    rules_with_names = []
    for antecedent_ids, consequent_ids in rules:
        antecedent_names = [movie_dict[id] for id in antecedent_ids if id in movie_dict]
        consequent_names = [movie_dict[id] for id in consequent_ids if id in movie_dict]
        rules_with_names.append((antecedent_names, consequent_names))
        print(antecedent_names, "-->", consequent_names)
    
    return rules_with_names

Finally, print your final association rules in the following format:

**movie_name_1, movie_name_2, ... --> movie_name_k**

where the movie names can be fetched by joining the movieId with the file 'movies.csv'. For example, one rule that you should find is:

**Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)**

**Hint: if you implement the algorith correctly, you will find 14 rules in total.**

**Hint: You may need to use the Pandas library to load and process the movies.csv file, such as use pandas.read_csv() to load the data. https://pandas.pydata.org/pandas-docs/dev/user_guide/10min.html is a good place to learn the basics about Pandas.**

In [26]:
# Set parameters
x = 3
min_support = 0.05
min_confidence = 0.8

# Load data into transactions
print("Loading movie rating data...")
data = load_data("movie_rated.txt")
print(f"Loaded {len(data)} user transactions\n")

# Run Apriori algorithm
print("Running Apriori Algorithm...")
frequent_itemsets = apriori_algorithm(data, x, min_support)

# Generate association rules
print("\nGenerating Association Rules...")
r = generate_rules(data, frequent_itemsets, min_confidence)

# Print rules with movie names
print("\n" + "="*70)
print("FINAL ASSOCIATION RULES (with Movie Names)")
print("="*70 + "\n")
rules_with_movie_names = rules_to_names(r, movie_dict)

print(f"\n{'='*70}")
print(f"Found {len(rules_with_movie_names)} association rules total")
print(f"{'='*70}")

Loading movie rating data...
Loaded 494 user transactions

Running Apriori Algorithm...

Processing Level 1 (itemsets of size 1)
Candidates of length 1 count: 408
After Pruning 1 count: 408
Frequent itemsets of length 1: 152

Processing Level 2 (itemsets of size 2)
Candidates of length 2 count: 11476
After Pruning 2 count: 11476
Frequent itemsets of length 2: 1846

Processing Level 3 (itemsets of size 3)
Candidates of length 3 count: 357760
After Pruning 3 count: 18113
Frequent itemsets of length 3: 8323

Algorithm Complete!
Total frequent itemsets found: 10321


Generating Association Rules...
Generating association rules from 10321 frequent itemsets...
Total rules with confidence >= 0.8: 5580


FINAL ASSOCIATION RULES (with Movie Names)

['Father of the Bride Part II (1995)'] --> ['Back to the Future (1985)']
['Friday (1995)'] --> ['Groundhog Day (1993)']
['Broken Arrow (1996)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Broken Arrow (1996)'] --> ['Jurassic Park (1993)']
['R